1: Install and Import Required Modules and Libraries

In [ ]:
# %pip install pandas numpy nltk scikit-learn
# %pip install -U bnlp_toolkit
# %pip install banglanlp

import pandas as pd
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from bnlp import BengaliCorpus as corpus # Import the BengaliCorpus class
from bnlp import NLTKTokenizer
#from banglanlp.tokenizer import Tokenizer

2: Load the newspaper dataset

In [3]:
# Load the CSV file
main_data = pd.read_csv("datasets/data.csv", encoding='utf-8')

# Display basic info and a sample of the data
main_data.info()
main_data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   content         4993 non-null   object
 1   category        5000 non-null   object
 2   author          5000 non-null   object
 3   published_date  5000 non-null   object
 4   title           5000 non-null   object
 5   url             5000 non-null   object
dtypes: object(6)
memory usage: 234.5+ KB


,content,category,author,published_date,title,url
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,bangladesh,গাজীপুর প্রতিনিধি,"০৪ জুলাই ২০১৩, ২৩:২৬","কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক...",http://www.prothom-alo.com/bangladesh/article/...
1,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,sports,অনলাইন ডেস্ক,"০৪ জুলাই ২০১৩, ২৩:০৯",সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,http://www.prothom-alo.com/sports/article/19028
2,জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...,bangladesh,নিজস্ব প্রতিবেদক,"০৪ জুলাই ২০১৩, ২২:২৫",সংসদে খালেদার অভিযোগের জবাব দিয়েছে ভারত,http://www.prothom-alo.com/bangladesh/article/...
3,সহজ পাসওয়ার্ডের কারণে অনলাইন অ্যাকাউন্ট সহজেই...,technology,অনলাইন ডেস্ক,"০৪ জুলাই ২০১৩, ২১:৩৭",পাসওয়ার্ড ভুলে যান!,http://www.prothom-alo.com/technology/article/...
4,কম্পিউটার মাউসের উদ্ভাবক ডগলাস অ্যাঙ্গেলবার্ট ...,technology,অনলাইন ডেস্ক,"০৪ জুলাই ২০১৩, ২১:৩৫",চলে গেলেন মাউস উদ্ভাবক,http://www.prothom-alo.com/technology/article/...
...,...,...,...,...,...,...
4995,গাজীপুরে বাংলাদেশ ধান গবেষণা ইনস্টিটিউটের (বিআ...,bangladesh,গাজীপুর প্রতিনিধি,"২৫ জুলাই ২০১৩, ০২:৪০","তিন বিজ্ঞানী বদলি, জড়িতদের বিরুদ্ধে ব্যবস্থা ...",http://www.prothom-alo.com/bangladesh/article/...
4996,পলাতক চৌধুরী মুঈনুদ্দীন ও আশরাফুজ্জামান খানের ...,bangladesh,নিজস্ব প্রতিবেদক,"২৫ জুলাই ২০১৩, ০২:৩৯",‘অধ্যাপক গিয়াসউদ্দিনকে বুকে অস্ত্র ঠেকিয়ে ধর...,http://www.prothom-alo.com/bangladesh/article/...
4997,স্বীকৃতিপ্রাপ্ত শিক্ষাপ্রতিষ্ঠান ঈদের আগে এমপি...,bangladesh,নিজস্ব প্রতিবেদক,"২৫ জুলাই ২০১৩, ০২:৩৮",ঈদের আগেই এমপিওভুক্তির দাবি,http://www.prothom-alo.com/bangladesh/article/...
4998,মেয়েটির বয়স এখন ১২। নাম আফরোজা। পড়ছে ষষ্ঠ শ...,bangladesh,"অমর সাহা, কলকাতা","২৫ জুলাই ২০১৩, ০২:৩৭",কোথায় যাবে আফরোজা,http://www.prothom-alo.com/bangladesh/article/...


3. Articles to sentence for grammar and spell checking

In [4]:
def bangla_articles_sentence(text):
    # Regular expression pattern for Bangla sentence boundaries (assuming sentences end with punctuation)
    sentences = re.split(r'(?<=[।?!])', text)
    return [sentence.strip() for sentence in sentences if sentence]

# Create a new DataFrame with one sentence per row
df = pd.DataFrame({
    'sentence': [sentence for article in main_data['content'].fillna('') for sentence in bangla_articles_sentence(article)]
})

df.info()
df
#print(df['sentence'].iloc[5])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101639 entries, 0 to 101638
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sentence  101639 non-null  object
dtypes: object(1)
memory usage: 794.2+ KB


,sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।
...,...
101634,চারটিতেই বিএনপি-সমর্থিত প্রার্থীরা জয়ী হন।
101635,এর গেজেট প্রকাশ করা হয় গত ২৪ জুন।
101636,আইনে গেজেট প্রকাশের এক মাসের মধ্যে শপথ গ্রহণের...
101637,তাই ২১ জুলাই তাঁদের শপথ পড়ানো হয়।


4: Text Cleaning and Normalization

In [5]:
# Count duplicate rows in the 'sentence' column
duplicate_count = df['sentence'].duplicated().sum()
print("Number of duplicate sentences:", duplicate_count)

# # Display duplicate rows in the 'sentence' column
# duplicate_sentences = df[df['sentence'].duplicated(keep=False)] \
#     .groupby('sentence').size().reset_index(name='count') \
#     .sort_values(by='count', ascending=False)
# # Display sentences with more than one occurrence
# print("Duplicate sentences and their counts:")
# print(duplicate_sentences[duplicate_sentences['count'] > 1])
#remove duplicate values
df.drop_duplicates(inplace=True)
print('Number of available rows after dropping duplicates:',df.shape[0])

Number of duplicate sentences: 6090
Number of available rows after dropping duplicates: 95549


In [6]:
# Count rows with only one word, one letter, only punctuation, or only whitespace
counts = {
    "one_word": df['sentence'].str.match(r'^[\u0980-\u09FF]+$|^\w+$').sum(),
    "one_letter": df['sentence'].str.match(r'^[\u0980-\u09FF]$|^[^\W\d_]$').sum(),
    "only_punctuation": df['sentence'].str.match(r'^[^\w\s]+$').sum(),
    "only_space": df['sentence'].str.match(r'^\s*$').sum(),
}
print("Counts:", counts)

# Drop rows in 'sentence' column that contain only one word, one letter, punctuation, or whitespace
df = df[~df['sentence'].str.match(r'^[\u0980-\u09FF]+$|^\w+$|^[\u0980-\u09FF]$|^[^\W\d_]$|^[^\w\s]+$|^\s*$', na=False)]

df

Counts: {'one_word': 7, 'one_letter': 0, 'only_punctuation': 8, 'only_space': 0}


,sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।
...,...
101631,বুধবার বিকেলে স্থানীয় সরকার বিভাগের সিটি করপো...
101633,"প্রসঙ্গত, গত ১৫ জুন এ চার সিটি করপোরেশনে মেয়র..."
101635,এর গেজেট প্রকাশ করা হয় গত ২৪ জুন।
101637,তাই ২১ জুলাই তাঁদের শপথ পড়ানো হয়।


In [7]:
def clean_text(text):
    bangla_punctuation = r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~।ঃ' #corpus.punctuations
    text = re.sub(r'[^\u0980-\u09FF\s' + bangla_punctuation + ']', '', text)  # Keep only Bangla characters with punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

df['cleaned_text'] = df['sentence'].apply(clean_text) 
df[['sentence', 'cleaned_text']].info()
df[['sentence', 'cleaned_text']].head()

<class 'pandas.core.frame.DataFrame'>
Index: 95534 entries, 0 to 101638
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sentence      95534 non-null  object
 1   cleaned_text  95534 non-null  object
dtypes: object(2)
memory usage: 2.2+ MB


,sentence,cleaned_text
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,এতে আতঙ্কিত হওয়ার কিছু নেই।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।


5: Tokenization

In [8]:
from collections import defaultdict

# Initialize tokenizer (mock example, replace NLTKTokenizer if using a specific library)
tokenizer = NLTKTokenizer()

# Tokenize text into words
def tokenize_text(text):
    tokens = tokenizer.word_tokenize(text)
    return tokens

# Build a vocabulary from the dataset
def build_vocab(sentences):
    vocab = defaultdict(lambda: len(vocab))  # Assign unique IDs to each word
    vocab['<PAD>'] = 0  # Reserve ID 0 for padding
    for sentence in sentences:
        for word in sentence:
            _ = vocab[word]
    return dict(vocab)

# Tokenize cleaned_text column
df['tokens'] = df['cleaned_text'].apply(tokenize_text)

# Build the vocabulary using tokens
vocab = build_vocab(df['tokens'])

# Convert tokens into numerical IDs
def convert_tokens_to_ids(tokens, vocab):
    return [vocab[word] for word in tokens]

df['token_id'] = df['tokens'].apply(lambda tokens: convert_tokens_to_ids(tokens, vocab))

# Create the lexicon_df with word-token mapping
lexicon_data = [{'word': word, 'token_id': token_id} for word, token_id in vocab.items()]
lexicon_df = pd.DataFrame(lexicon_data)

df[['cleaned_text', 'tokens', 'token_id']].head()
# lexicon_df[['word', 'token_id']].head()

,cleaned_text,tokens,token_id
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[গাজীপুরের, কালিয়াকৈর, উপজেলার, তেলিরচালা, এল...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[এ, ঘটনায়, বিক্ষোভ, করেছেন, ওই, কারখানার, শ্র...","[20, 21, 22, 23, 24, 13, 25, 19]"
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[সফিপুর, মডার্ন, হাসপাতালের, জরুরি, বিভাগের, চ...","[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 3..."
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[এতে, আতঙ্কিত, হওয়ার, কিছু, নেই, ।]","[44, 45, 46, 47, 48, 19]"
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[অসুস্থদের, চিকিত্সা, দেওয়া, হয়েছে, ।]","[49, 50, 51, 52, 19]"


6. Add Bangla POS Tagger

In [9]:
from bnlp import BengaliPOS

# Instantiate the BengaliPOS tagger
bn_pos = BengaliPOS()

# Apply POS tagging: join tokens into a sentence before passing to bn_pos.tag()
df['pos_tags'] = df['cleaned_text'].apply(lambda tokens: bn_pos.tag(tokens))

# Display the DataFrame with POS tags
df[['cleaned_text', 'pos_tags']].head()

,cleaned_text,pos_tags
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[(গাজীপুরের, NC), (কালিয়াকৈর, NC), (উপজেলার, ..."
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[(এ, DAB), (ঘটনায়, NC), (বিক্ষোভ, NC), (করেছে..."
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[(সফিপুর, NP), (মডার্ন, NC), (হাসপাতালের, NC),..."
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[(এতে, PPR), (আতঙ্কিত, JJ), (হওয়ার, NC), (কিছ..."
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[(অসুস্থদের, NC), (চিকিত্সা, NC), (দেওয়া, NV)..."


7. Generate incorrect sentence for grammar checking

In [10]:
import random
# Function to introduce a simple grammatical error by shuffling words
def introduce_errors(sentence):
    # Split sentence into words and punctuation, preserving spaces
    words_and_spaces = re.split(r'(\s+)', sentence)  # Split on spaces, keeping spaces in the result
    # Extract only Bangla words
    bangla_words = [word for word in words_and_spaces if re.match(r'^[\u0980-\u09FF]+$', word)]
    
    if len(bangla_words) > 1:
        # Shuffle words or remove one randomly to simulate error
        if random.choice([True, False]):
            random.shuffle(bangla_words)
        else:
            bangla_words.pop(random.randint(0, len(bangla_words) - 1))
    
    # Replace Bangla words in the original structure
    result = []
    bangla_word_index = 0
    for token in words_and_spaces:
        if re.match(r'^[\u0980-\u09FF]+$', token):  # If it's a Bangla word
            if bangla_word_index < len(bangla_words):
                result.append(bangla_words[bangla_word_index])
                bangla_word_index += 1
            else:
                result.append('')  # This handles removed words
        else:
            result.append(token)  # Preserve spaces and punctuation
    return ''.join(result)

# Apply the function to generate incorrect sentences
df['incorrect_sentence'] = df['cleaned_text'].apply(introduce_errors)
df[['cleaned_text', 'incorrect_sentence']]

,cleaned_text,incorrect_sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,এ ঘটনায় বিক্ষোভ ওই কারখানার শ্রমিকেরা।
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের আল আমিন...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,এতে কিছু হওয়ার আতঙ্কিত নেই।
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,চিকিত্সা দেওয়া হয়েছে।
...,...,...
101631,বুধবার বিকেলে স্থানীয় সরকার বিভাগের সিটি করপো...,স্থানীয় বিকেলে সিটি বিভাগের মেয়রের শাখা বুধব...
101633,"প্রসঙ্গত, গত ১৫ জুন এ চার সিটি করপোরেশনে মেয়র...","প্রসঙ্গত, গত এ ১৫ চার করপোরেশনে সিটি মেয়র জুন..."
101635,এর গেজেট প্রকাশ করা হয় গত ২৪ জুন।,এর গেজেট প্রকাশ করা হয় ২৪ জুন।
101637,তাই ২১ জুলাই তাঁদের শপথ পড়ানো হয়।,তাঁদের তাই শপথ জুলাই পড়ানো ২১ হয়।


In [11]:
# print(df['sentence'].iloc[5])
# print(df['incorrect_sentence'].iloc[5])

8. Generate Synthetic Errors for the spell correction phase

In [22]:
import random

# Function to introduce synthetic spelling errors
def introduce_spelling_errors(word):
    """Randomly introduce an error into the given word."""
    if len(word) < 2:
        return word, 0  # Skip very short words, no change
    
    error_type = random.choice(['insert', 'delete', 'substitute', 'transpose'])
    original_word = word
    changed = 1  # Tracks if a word was changed

    if error_type == 'insert':
        pos = random.randint(0, len(word))
        char = chr(random.randint(2432, 2559))  # Bangla Unicode range
        word = word[:pos] + char + word[pos:]
    elif error_type == 'delete':
        pos = random.randint(0, len(word) - 1)
        word = word[:pos] + word[pos + 1:]
    elif error_type == 'substitute':
        pos = random.randint(0, len(word) - 1)
        char = chr(random.randint(2432, 2559))
        word = word[:pos] + char + word[pos + 1:]
    elif error_type == 'transpose' and len(word) > 1:
        pos = random.randint(0, len(word) - 2)
        word = word[:pos] + word[pos + 1] + word[pos] + word[pos + 2:]
    else:
        changed = 0  # No change if transpose fails on a single-character word

    if word == original_word:
        changed = 0  # No change occurred

    return word, changed

# Apply spelling errors to tokens and calculate attention_mask
def apply_errors(tokens):
    result = [introduce_spelling_errors(word) for word in tokens]
    words, attention_mask = zip(*result)
    return words, list(attention_mask)

df[['synth_spelling_errors', 'attention_mask']] = df['tokens'].apply(
    lambda tokens: pd.Series(apply_errors(tokens))
)

# Convert tokens to strings
df['synth_spelling_errors'] = df['synth_spelling_errors'].apply(lambda tokens: ' '.join(tokens))

# Prepare the bert DataFrame
bert_rows = []
for _, row in df.iterrows():
    # Add row for original cleaned_text
    bert_rows.append({
        "sentence": row['sentence'],
        "attention_mask": [0] * len(row['tokens']),  # All 0 for original text
        "label": 0
    })
    # Add row for synthetic spelling errors
    bert_rows.append({
        "sentence": row['synth_spelling_errors'],
        "attention_mask": row['attention_mask'],
        "label": 1
    })

# Create final DataFrame
bert_df = pd.DataFrame(bert_rows)

bert_df.head(8)

,sentence,attention_mask,label
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,গাজীপঌরের কালযি়াকৈর ধউপজেলার তেলিরালা এলাকা়য...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[0, 0, 0, 0, 0, 0, 0, 0]",0
3,এ ঘটনা় বিক্ষখোভ ৥রেছেন ঢ়ই কারখানার৽ ্শরমিকেরা ।,"[0, 1, 1, 1, 1, 1, 1, 0]",1
4,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
5,সফিুর মডর্ন হাসপাতাের জুরি িবভাগের চিকিত্সথ আ৫...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...",1
6,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[0, 0, 0, 0, 0, 0]",0
7,এওে আঙত্কিত হয়ার কছিু নই ।,"[1, 1, 1, 1, 1, 0]",1


In [ ]:
# # Initialize tokenizer (mock example, replace NLTKTokenizer if using a specific library)
# tokenizer = NLTKTokenizer()

# # Tokenize and convert to integer token IDs
# def tokenize_and_convert_to_ids(sentence, vocab):
#     tokens = tokenizer.word_tokenize(sentence)  # Tokenize the sentence
#     token_ids = [int(vocab[word]) if word in vocab else int(vocab['<UNK>']) for word in tokens]  # Map and ensure integer
#     return token_ids

# # Add a <UNK> token to the vocabulary for unknown words
# if '<UNK>' not in vocab:
#     vocab['<UNK>'] = len(vocab)

# # Add the token_id column to bert_df
# bert_df['token_id'] = bert_df['sentence'].apply(lambda x: tokenize_and_convert_to_ids(x, vocab))

# # Preview the updated DataFrame
# bert_df.head(8)

,sentence,attention_mask,label,token_id
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,গাজীপ৳ুরের ালিয়াকৈর উপজেলংার তেলিচরালা এলাকা়...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",0,"[84488, 84488, 84488, 84488, 84488, 84488, 844..."
2,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[1, 1, 1, 1, 1, 1, 1, 1]",1,"[20, 21, 22, 23, 24, 13, 25, 19]"
3,এ ঘনায় বিক্ষভো রেছেন ই কারখানর ঳শ্রমিকেরা ।,"[1, 0, 0, 0, 0, 0, 0, 1]",0,"[20, 84488, 84488, 84488, 2044, 84488, 84488, 19]"
4,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1,"[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 3..."
5,সফিপু঍র মডা৬্ন হাসপাতলের জরুির বিভােগর চিকিত্স...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0,"[84488, 84488, 84488, 84488, 84488, 84488, 844..."
6,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[1, 1, 1, 1, 1, 1]",1,"[44, 45, 46, 47, 48, 19]"
7,এত৬ ঩আতঙ্কিত ওহয়ার কিছ েই ।,"[0, 0, 0, 0, 0, 1]",0,"[84488, 84488, 84488, 84488, 84488, 19]"


In [23]:
bert_df[['sentence', 'attention_mask', 'label']].to_csv("datasets/bert_data.csv", index=False)
print("Preprocessed bert data saved to 'datasets/bert_data.csv'")

bert_df

Preprocessed bert data saved to 'datasets/bert_data.csv'


,sentence,attention_mask,label
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,গাজীপঌরের কালযি়াকৈর ধউপজেলার তেলিরালা এলাকা়য...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
2,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[0, 0, 0, 0, 0, 0, 0, 0]",0
3,এ ঘটনা় বিক্ষখোভ ৥রেছেন ঢ়ই কারখানার৽ ্শরমিকেরা ।,"[0, 1, 1, 1, 1, 1, 1, 0]",1
4,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...
191063,রএ গেজে ৴্রকাশ রা হ় গ ঞ৪ ুন ।,"[1, 1, 1, 1, 1, 1, 1, 1, 0]",1
191064,তাই ২১ জুলাই তাঁদের শপথ পড়ানো হয়।,"[0, 0, 0, 0, 0, 0, 0, 0]",0
191065,ত৅ই ়২১ ুজলাই তাঁঅদের পশথ োড়ানো যহ় ।,"[1, 1, 1, 1, 1, 1, 1, 0]",1
191066,"স্থানীয় সরকার (সিটি করপোরেশন আইন), ২০০৯-এর ৬ ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


9. Double Metaphone-inspired phonetic encoding function for Bangla

In [14]:
"""Generate a phonetic encoding for Bangla words based on specified rules."""
def bangla_phonetic_encoding(word):
    phonetic_map = {
        # Special Characters and Signs
        '্': '',   # Virama/Hasant (code 0)
        'ঁ': '',   # Candrabindu (code 0)
        'ং': 'N',  # Anusvara
        'ঃ': 'H',  # Visarga

        # Vowels and Signs
        'অ': 'A', 'আ': 'AA', 'া': 'AA', 
        'ই': 'I', 'ি': 'I', 'ঈ': 'II', 'ী': 'II',
        'উ': 'U', 'ু': 'U', 'ঊ': 'UU', 'ূ': 'UU',
        'ঋ': 'RI', 'র': 'R', 'ৃ': 'RI',
        'এ': 'E', 'ে': 'E', 'ঐ': 'OI', 'ৈ': 'OI',
        'ও': 'O', 'ো': '', 'ঔ': 'OU', 'ৌ': 'OU',

        # Consonants
        'ক': 'K', 'খ': 'KH', 'গ': 'G', 'ঘ': 'GH', 'ঙ': 'NG',
        'চ': 'C', 'ছ': 'CH', 'জ': 'J', 'ঝ': 'JH', 'য': 'Y',
        'ট': 'T', 'ঠ': 'TH', 'ড': 'D', 'ঢ': 'DH', 'ণ': 'N',
        'ত': 'T', 'থ': 'TH', 'দ': 'D', 'ধ': 'DH', 'ন': 'N',
        'প': 'P', 'ফ': 'PH', 'ব': 'B', 'ভ': 'BH', 'ম': 'M',
        'শ': 'SH', 'ষ': 'SH', 'স': 'S', 'হ': 'H',

        # Additional Characters
        'ড়': 'RH', 'ঢ়': 'RH', 'য়': 'Y', 'ঞ': 'NY',
        'ল': 'L', 'ৠ': 'RR', '০': '0', '১': '1', 
        '২': '2', '৩': '3', '৪': '4', '৫': '5', '৬': '6', 
        '৭': '7', '৮': '8', '৯': '9'
    }
    
    encoded_word = []
    
    for char in word:
        if char in phonetic_map:
            encoded_word.append(phonetic_map[char])
    
    # Join encoded characters to form the phonetic encoding
    return ''.join(encoded_word)

# Apply phonetic encoding to each word in the tokens column
df['phonetic_tokens'] = df['tokens'].apply(
    lambda tokens: [bangla_phonetic_encoding(word) for word in tokens]
)

# Convert the phonetic tokens back to a sentence for easier reading
df['phonetic_encoded_sentence'] = df['phonetic_tokens'].apply(
    lambda tokens: ' '.join(tokens)
)

# Display the first few rows
df[['sentence', 'phonetic_tokens', 'phonetic_encoded_sentence']].head()

,sentence,phonetic_tokens,phonetic_encoded_sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[GAAJIIPURER, KAALIYAAKOIR, UPJELAAR, TELIRCAA...",GAAJIIPURER KAALIYAAKOIR UPJELAAR TELIRCAALAA ...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[E, GHTNAAY, BIKSHBH, KRECHEN, OI, KAARKHAANAA...",E GHTNAAY BIKSHBH KRECHEN OI KAARKHAANAAR SHRM...
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[SPHIPUR, MDAARN, HAASPAATAALER, JRURI, BIBHAA...",SPHIPUR MDAARN HAASPAATAALER JRURI BIBHAAGER C...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[ETE, AATNGKIT, HOYAAR, KICHU, NEI, ]",ETE AATNGKIT HOYAAR KICHU NEI
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[ASUSTHDER, CIKITSAA, DEOYAA, HYECHE, ]",ASUSTHDER CIKITSAA DEOYAA HYECHE


10: Save Preprocessed Sentence Data

In [15]:
df[['sentence', 'cleaned_text', 'tokens','pos_tags',	'incorrect_sentence',	'synth_spelling_errors',	'phonetic_tokens',	'phonetic_encoded_sentence']].to_csv("datasets/sentence_data.csv", index=False)
print("Preprocessed sentence data saved to 'datasets/sentence_data.csv'")

df

Preprocessed sentence data saved to 'datasets/sentence_data.csv'


,sentence,cleaned_text,tokens,token_id,pos_tags,incorrect_sentence,synth_spelling_errors,attention_mask,phonetic_tokens,phonetic_encoded_sentence
0,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,"[গাজীপুরের, কালিয়াকৈর, উপজেলার, তেলিরচালা, এল...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[(গাজীপুরের, NC), (কালিয়াকৈর, NC), (উপজেলার, ...",বৃহস্পতিবার একটি গাজীপুরের কালিয়াকৈর এলাকায় ...,ঀগাজীপুরের কালিয়া঳কৈর উপেলার তেলিরালা এলকায় ...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[GAAJIIPURER, KAALIYAAKOIR, UPJELAAR, TELIRCAA...",GAAJIIPURER KAALIYAAKOIR UPJELAAR TELIRCAALAA ...
1,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,এ ঘটনায় বিক্ষোভ করেছেন ওই কারখানার শ্রমিকেরা।,"[এ, ঘটনায়, বিক্ষোভ, করেছেন, ওই, কারখানার, শ্র...","[20, 21, 22, 23, 24, 13, 25, 19]","[(এ, DAB), (ঘটনায়, NC), (বিক্ষোভ, NC), (করেছে...",বিক্ষোভ করেছেন কারখানার এ ওই ঘটনায় শ্রমিকেরা।,এ টনায় বিকষোভ ৛রেছেন ইও কারখানঠর শ্রমিকরো ।,"[0, 1, 1, 1, 1, 1, 1, 0]","[E, GHTNAAY, BIKSHBH, KRECHEN, OI, KAARKHAANAA...",E GHTNAAY BIKSHBH KRECHEN OI KAARKHAANAAR SHRM...
2,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,সফিপুর মডার্ন হাসপাতালের জরুরি বিভাগের চিকিত্স...,"[সফিপুর, মডার্ন, হাসপাতালের, জরুরি, বিভাগের, চ...","[26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 3...","[(সফিপুর, NP), (মডার্ন, NC), (হাসপাতালের, NC),...",জরুরি তাঁরা অসুস্থ চিকিত্সক আলো হাসপাতালের হয়...,সফিপর মডারন্ ধাসপাতালের রজুরি বিঢ়ভাগের িকিত্সক...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[SPHIPUR, MDAARN, HAASPAATAALER, JRURI, BIBHAA...",SPHIPUR MDAARN HAASPAATAALER JRURI BIBHAAGER C...
3,এতে আতঙ্কিত হওয়ার কিছু নেই।,এতে আতঙ্কিত হওয়ার কিছু নেই।,"[এতে, আতঙ্কিত, হওয়ার, কিছু, নেই, ।]","[44, 45, 46, 47, 48, 19]","[(এতে, PPR), (আতঙ্কিত, JJ), (হওয়ার, NC), (কিছ...",এতে আতঙ্কিত হওয়ার নেই।,এেত আতঙ্কধিত ওয়ার কিৌছু নইে ।,"[1, 1, 1, 1, 1, 0]","[ETE, AATNGKIT, HOYAAR, KICHU, NEI, ]",ETE AATNGKIT HOYAAR KICHU NEI
4,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,অসুস্থদের চিকিত্সা দেওয়া হয়েছে।,"[অসুস্থদের, চিকিত্সা, দেওয়া, হয়েছে, ।]","[49, 50, 51, 52, 19]","[(অসুস্থদের, NC), (চিকিত্সা, NC), (দেওয়া, NV)...",দেওয়া অসুস্থদের চিকিত্সা হয়েছে।,অসুস্দথের চিকিতসা দেওয় হ়েছে ।,"[1, 1, 1, 1, 0]","[ASUSTHDER, CIKITSAA, DEOYAA, HYECHE, ]",ASUSTHDER CIKITSAA DEOYAA HYECHE
...,...,...,...,...,...,...,...,...,...,...
101631,বুধবার বিকেলে স্থানীয় সরকার বিভাগের সিটি করপো...,বুধবার বিকেলে স্থানীয় সরকার বিভাগের সিটি করপো...,"[বুধবার, বিকেলে, স্থানীয়, সরকার, বিভাগের, সিট...","[1532, 975, 1490, 1774, 30, 418, 419, 1785, 14...","[(বুধবার, NC), (বিকেলে, NC), (স্থানীয়, JJ), (...",বুধবার বিকেলে স্থানীয় সরকার বিভাগের সিটি করপো...,বুধার বিক঎লে স্থানযী় সকরার বিভাগে সিৗি ৊করপোর...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[BUDHBAAR, BIKELE, STHAANIIY, SRKAAR, BIBHAAGE...",BUDHBAAR BIKELE STHAANIIY SRKAAR BIBHAAGER SIT...
101633,"প্রসঙ্গত, গত ১৫ জুন এ চার সিটি করপোরেশনে মেয়র...","প্রসঙ্গত, গত ১৫ জুন এ চার সিটি করপোরেশনে মেয়র...","[প্রসঙ্গত, ,, গত, ১৫, জুন, এ, চার, সিটি, করপোর...","[4824, 38, 373, 2575, 375, 20, 214, 418, 6327,...","[(প্রসঙ্গত, JJ), (,, PU), (গত, JJ), (১৫, NP), ...","প্রসঙ্গত, গত ১৫ জুন এ সিটি করপোরেশনে মেয়র নির...","প্রসঙগ্ত , তগ ৫১ জঅুন এ চরা তিটি করপোজেশনে মে৒...","[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0]","[PRSNGGT, , GT, 15, JUN, E, CAAR, SITI, KRPRES...",PRSNGGT GT 15 JUN E CAAR SITI KRPRESHNE MEYR ...
101635,এর গেজেট প্রকাশ করা হয় গত ২৪ জুন।,এর গেজেট প্রকাশ করা হয় গত ২৪ জুন।,"[এর, গেজেট, প্রকাশ, করা, হয়, গত, ২৪, জুন, ।]","[424, 4380, 1175, 173, 67, 373, 3203, 375, 19]","[(এর, PPR), (গেজেট, NC), (প্রকাশ, NC), (করা, N...",এর গেজেট প্রকাশ করা হয় ২৪ জুন।,রএ েগজেট প্রাকশ কর ংয় ৅ত ৪২ জু০ ।,"[1, 1, 1, 1, 1, 1, 1, 1, 0]","[ER, GEJET, PRKAASH, KRAA, HY, GT, 24, JUN, ]",ER GEJET PRKAASH KRAA HY GT 24 JUN
101637,তাই ২১ জুলাই তাঁদের শপথ পড়ানো হয়।,তাই ২১ জুলাই তাঁদের শপথ পড়ানো হয়।,"[তাই, ২১, জুলাই, তাঁদের, শপথ, পড়ানো, হয়, ।]","[452, 1651, 874, 572, 3809, 55842, 67, 19]","[(তাই, CSB), (২১, RDF), (জুলাই, NC), (তাঁদের,

Lexicon

In [16]:
import string
# Function to split text into words and remove punctuation
def split_and_clean(sentence):
    # Remove extra spaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()  # Remove extra spaces
    bangla_punctuation = "।!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"  #corpus.punctuations
    all_punctuation = string.punctuation + bangla_punctuation
    # Remove punctuation and split into words
    words = sentence.translate(str.maketrans('', '', all_punctuation)).split()
    return words

# Split sentences into words and explode them into a new DataFrame
df_words = df["cleaned_text"].apply(split_and_clean).explode().reset_index(drop=True)
df_words = df_words.to_frame(name="word")

print(df_words)

               word
0         গাজীপুরের
1        কালিয়াকৈর
2           উপজেলার
3         তেলিরচালা
4           এলাকায়
...             ...
1247096         দিন
1247097     পর্যন্ত
1247098     অপেক্ষা
1247099        করতে
1247100         হবে

[1247101 rows x 1 columns]


In [17]:
# Count duplicate rows in the 'sentence' column
duplicate_count = df_words['word'].duplicated().sum()
print("Number of duplicate sentences:", duplicate_count)

#remove duplicate values
df_words.drop_duplicates(inplace=True)

print('Number of available rows after dropping duplicates:',df_words.shape[0])

Number of duplicate sentences: 1158130
Number of available rows after dropping duplicates: 88971


In [18]:
from collections import Counter

# Initialize the lexicon
lexicon = {}

# Generate the lexicon
for index, row in df.iterrows():
    tokens = row['tokens']
    pos_tags = row['pos_tags']
    
    # Skip rows with mismatched lengths or empty values
    if not tokens or not pos_tags or len(tokens) != len(pos_tags):
        # print(f"Skipping row {index} due to mismatch or empty data.")
        continue

    for i, token in enumerate(tokens):
        if token not in lexicon:
            lexicon[token] = {
                "phonetic_encoding": row['phonetic_tokens'][i],
                "pos": [pos_tags[i][1]],
                "frequency": 1,
                "common_errors": []
            }
        else:
            if pos_tags[i][1] not in lexicon[token]["pos"]:
                lexicon[token]["pos"].append(pos_tags[i][1])
            lexicon[token]["frequency"] += 1

# Identify common errors
for index, row in df.iterrows():
    errors = row['synth_spelling_errors'].split()  # Assuming errors are space-separated
    for error in errors:
        correct_word = row['tokens'][0]  # Assuming the first token is the correct one
        if correct_word in lexicon:
            lexicon[correct_word]["common_errors"].append(error)

# Convert to DataFrame for easier handling
lexicon_df = pd.DataFrame.from_dict(lexicon, orient='index')
lexicon_df.to_csv("datasets/bangla_lexicon.csv")
lexicon_df

,phonetic_encoding,pos,frequency,common_errors
গাজীপুরের,GAAJIIPURER,"[NC, NP]",144,"[ঀগাজীপুরের, কালিয়া঳কৈর, উপেলার, তেলিরালা, এল..."
কালিয়াকৈর,KAALIYAAKOIR,"[NC, NP]",22,"[কালিয়াৈকর, পরিবপন, শ্কমিক, সমিকির, সাধাণর, ৫..."
উপজেলার,UPJELAAR,"[NC, NP]",1055,"[উপঐেলার, পিআই঴, সাইদ঳ুর, হরমান, জানন, ,, াকজে..."
তেলিরচালা,TELIRCAALAA,[NC],5,[]
এলাকায়,ELAAKAAY,"[NC, JQ, NP, VM, VAUX, ALC, JJ, DAB]",887,"[এলাকায৛়, আামর, জনপ্রযি়তা, ও, বৈজয়, সুনশ্চি..."
...,...,...,...,...
তথ্যচিত্রও,TTHYCITRO,[NC],1,[]
জালান,JAALAAN,[VM],1,[]
শুভ্রজিৎ,SHUBHRJI,[NP],1,[]
তথ্যচিত্রে,TTHYCITRE,[NC],1,"[থত্যচিত্রে, একই, হোমে৑, থাকো, রআও, তিনি, কিশো..."


Check the basic statistics of the corpus

In [19]:
# Check length of preprocessed text
df['cleaned_text'].apply(lambda x: len(x.split())).describe()

count    95534.000000
mean        13.058618
std         12.578634
min          1.000000
25%          8.000000
50%         11.000000
75%         16.000000
max       1698.000000
Name: cleaned_text, dtype: float64

In [20]:
# from nltk.tokenize import sent_tokenize

# # Number of tokens in each row
# df['num_tokens'] = df['tokens'].apply(len)

# # Number of sentences in each row
# df['num_sentences'] = df['sentence'].apply(lambda text: len(sent_tokenize(str(text))))

# # Total number of tokens in the dataset
# total_tokens = df['num_tokens'].sum()
# print("Total Tokens:", total_tokens)

# # Total number of sentences in the dataset
# total_sentences = df['num_sentences'].sum()
# print("Total Sentences:", total_sentences)

# # Summary of token count per row
# print("Token Count per Row Summary:")
# print(df['num_tokens'].describe())

# # Summary of sentence count per row
# print("Sentence Count per Row Summary:")
# print(df['num_sentences'].describe())